# Pub/Sub Patterns: Broadcaster & EventBus

Two complementary event patterns:
- **Broadcaster**: Singleton, global channel, O(1) memory overhead
- **EventBus**: Instance-based, topic routing, concurrent handlers

In [1]:
# Setup
import asyncio

from lionpride.core.broadcaster import Broadcaster
from lionpride.core.event import Event
from lionpride.core.eventbus import EventBus

## Part 1: Broadcaster - Singleton Pub/Sub

Singleton pattern with class-level subscriptions. All instances share subscribers.

In [2]:
# Define event type and broadcaster subclass
class ShutdownEvent(Event):
    event_type: str = "shutdown"
    reason: str = ""


class ShutdownBroadcaster(Broadcaster):
    _event_type = ShutdownEvent
    _subscribers = []  # Class-level state
    _instance = None

### Demo 1: Subscribe & Broadcast

In [3]:
# Singleton pattern - same instance every time
b1 = ShutdownBroadcaster()
b2 = ShutdownBroadcaster()
print(f"Same instance: {b1 is b2}")  # True

# Subscribe callbacks (stored at class level)
received = []


def handler1(event):
    received.append(f"Handler1: {event.reason}")


def handler2(event):
    received.append(f"Handler2: {event.reason}")


ShutdownBroadcaster.subscribe(handler1)
ShutdownBroadcaster.subscribe(handler2)
print(f"Subscribers: {ShutdownBroadcaster.get_subscriber_count()}")  # 2

# Broadcast event
event = ShutdownEvent(reason="maintenance")
await ShutdownBroadcaster.broadcast(event)

print("\nReceived:")
for msg in received:
    print(f"  {msg}")

Same instance: True
Subscribers: 2

Received:
  Handler1: maintenance
  Handler2: maintenance


### Demo 2: Mixed Sync/Async Callbacks

In [4]:
# Clear previous subscribers
ShutdownBroadcaster._subscribers.clear()
results = []


# Sync callback
def sync_handler(event):
    results.append(f"sync: {event.reason}")


# Async callback
async def async_handler(event):
    await asyncio.sleep(0.01)  # Simulate async work
    results.append(f"async: {event.reason}")


ShutdownBroadcaster.subscribe(sync_handler)
ShutdownBroadcaster.subscribe(async_handler)

# Both execute correctly
await ShutdownBroadcaster.broadcast(ShutdownEvent(reason="upgrade"))

print("Results:")
for r in results:
    print(f"  {r}")

Results:
  sync: upgrade
  async: upgrade


### Demo 3: Weakref Auto-Cleanup

In [5]:
ShutdownBroadcaster._subscribers.clear()


# Create object with callback method (bound method)
class Handler:
    def __init__(self, name):
        self.name = name
        self.count = 0

    def on_event(self, event):
        self.count += 1
        print(f"{self.name} received event #{self.count}")


# Subscribe bound method
handler_obj = Handler("ServiceA")
ShutdownBroadcaster.subscribe(handler_obj.on_event)
print(f"Subscribers after subscribe: {ShutdownBroadcaster.get_subscriber_count()}")  # 1

# Broadcast works
await ShutdownBroadcaster.broadcast(ShutdownEvent(reason="test1"))

# Delete object - weakref auto-cleanup on next operation
del handler_obj
import gc

gc.collect()

# Next operation cleans up dead weakrefs
count = ShutdownBroadcaster.get_subscriber_count()
print(f"Subscribers after GC: {count}")  # Note: May still be 1 due to GC timing

Subscribers after subscribe: 1
ServiceA received event #1
Subscribers after GC: 1


### Demo 4: Exception Isolation

In [6]:
ShutdownBroadcaster._subscribers.clear()
success_log = []


def failing_handler(event):
    raise RuntimeError("Handler crashed!")


def working_handler(event):
    success_log.append(f"Working handler got: {event.reason}")


ShutdownBroadcaster.subscribe(failing_handler)
ShutdownBroadcaster.subscribe(working_handler)

# Broadcast continues despite exception (logged, not raised)
await ShutdownBroadcaster.broadcast(ShutdownEvent(reason="safe"))

print(f"Working handler still executed: {success_log}")
print("No exception raised to caller (fire-and-forget)")

Error in subscriber callback: Handler crashed!
Traceback (most recent call last):
  File "/Users/lion/projects/open-source/lionpride/src/lionpride/core/broadcaster.py", line 92, in broadcast
    callback(event)
  File "/var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/ipykernel_90167/4017429955.py", line 6, in failing_handler
    raise RuntimeError("Handler crashed!")
RuntimeError: Handler crashed!


Working handler still executed: ['Working handler got: safe']
No exception raised to caller (fire-and-forget)


## Part 2: EventBus - Topic-Based Routing

Instance-based pub/sub with topic filtering and concurrent handler execution.

### Demo 1: Topic Subscription & Emit

In [7]:
bus = EventBus()
events = []


async def log_start(node_id: str):
    events.append(f"START: {node_id}")


async def log_complete(node_id: str, duration: float):
    events.append(f"COMPLETE: {node_id} ({duration}s)")


# Subscribe to different topics
bus.subscribe("node.start", log_start)
bus.subscribe("node.complete", log_complete)

print(f"Topics: {bus.topics()}")  # ['node.start', 'node.complete']
print(f"Handlers for 'node.start': {bus.handler_count('node.start')}")  # 1

# Emit events - handlers only receive matching topics
await bus.emit("node.start", node_id="n1")
await bus.emit("node.complete", node_id="n1", duration=0.5)
await bus.emit("node.start", node_id="n2")

print("\nEvents:")
for e in events:
    print(f"  {e}")

Topics: ['node.start', 'node.complete']
Handlers for 'node.start': 1

Events:
  START: n1
  COMPLETE: n1 (0.5s)
  START: n2


### Demo 2: Multiple Handlers - Concurrent Execution

In [8]:
bus = EventBus()
execution_order = []


async def slow_handler():
    execution_order.append("slow_start")
    await asyncio.sleep(0.02)
    execution_order.append("slow_end")


async def fast_handler():
    execution_order.append("fast")


# Both subscribe to same topic
bus.subscribe("test", slow_handler)
bus.subscribe("test", fast_handler)

# Handlers run concurrently via asyncio.gather()
await bus.emit("test")

print("Execution order (concurrent):")
for step in execution_order:
    print(f"  {step}")
print("\nNote: 'fast' completes before 'slow_end' (concurrent execution)")

Execution order (concurrent):
  slow_start
  fast
  slow_end

Note: 'fast' completes before 'slow_end' (concurrent execution)


### Demo 3: Exception Isolation

In [9]:
bus = EventBus()
results = []


async def failing_handler(value: int):
    raise ValueError(f"Handler failed on {value}")


async def working_handler(value: int):
    results.append(f"Success: {value}")


bus.subscribe("test", failing_handler)
bus.subscribe("test", working_handler)

# Emit doesn't raise - exceptions suppressed via gather(return_exceptions=True)
await bus.emit("test", value=42)

print(f"Working handler executed: {results}")
print("Failing handler exception suppressed (no propagation)")

Working handler executed: ['Success: 42']
Failing handler exception suppressed (no propagation)


### Demo 4: Metrics Collection Pattern

In [10]:
bus = EventBus()
metrics = {"requests": 0, "errors": 0, "total_duration": 0.0}


# Metrics handlers
async def count_request(**kwargs):
    metrics["requests"] += 1


async def count_error(**kwargs):
    metrics["errors"] += 1


async def track_duration(duration: float, **kwargs):
    metrics["total_duration"] += duration


# Cross-cutting logger
logs = []


async def log_all(event_type: str, **kwargs):
    logs.append(f"[LOG] {event_type}: {kwargs}")


# Subscribe to multiple topics
bus.subscribe("request.start", count_request)
bus.subscribe("request.start", log_all)
bus.subscribe("request.complete", track_duration)
bus.subscribe("request.complete", log_all)
bus.subscribe("request.error", count_error)
bus.subscribe("request.error", log_all)

# Simulate request lifecycle
await bus.emit("request.start", event_type="start", request_id="r1")
await bus.emit("request.complete", event_type="complete", request_id="r1", duration=0.15)
await bus.emit("request.start", event_type="start", request_id="r2")
await bus.emit("request.error", event_type="error", request_id="r2", error="timeout")

print("Metrics:")
for k, v in metrics.items():
    print(f"  {k}: {v}")
print(f"\nLogs ({len(logs)} entries):")
for log in logs:
    print(f"  {log}")

Metrics:
  requests: 2
  errors: 1
  total_duration: 0.15

Logs (4 entries):
  [LOG] start: {'request_id': 'r1'}
  [LOG] complete: {'request_id': 'r1', 'duration': 0.15}
  [LOG] start: {'request_id': 'r2'}
  [LOG] error: {'request_id': 'r2', 'error': 'timeout'}


## Summary

### Broadcaster
- **Pattern**: Singleton, class-level subscriptions
- **Use case**: Global event channel (shutdown, config changes)
- **Memory**: O(1) overhead per subclass
- **Cleanup**: Weakref auto-cleanup on GC
- **Trade-offs**: Simple API, less flexible (single channel)

### EventBus
- **Pattern**: Instance-based, topic routing
- **Use case**: Observability (metrics, tracing, logging)
- **Concurrency**: Handlers run concurrently via `gather()`
- **Exception**: Isolated via `return_exceptions=True`
- **Trade-offs**: Flexible routing, per-instance overhead

### When to Use
- **Broadcaster**: Application-wide notifications, minimal overhead
- **EventBus**: Multi-topic observability, handler isolation required